In [ ]:
#  Hidden Setup
import pandas as pd
import openai
from credentials import openai_api_key

# Read the first CSV file
file_path = '/Users/mkarreth/Downloads/comp.csv'
df1 = pd.read_csv(file_path)

# Read the second CSV file
df2 = pd.read_csv('/Users/mkarreth/Downloads/prod.csv')

# Set up OpenAI API credentials
openai.api_key = openai_api_key

# A simple LLM-Lookup
It's just the good old Excel VLOOKUP, but powered by LLMs. Specifically ever wondered about that approximate match functionality in a VLOOKUP? Well here we go for real.

## (1) Take two CSV files and read them into dataframes

In [ ]:
companies = df1
website_products = df2
display(companies.describe())
display(website_products.describe())

In [ ]:
display(companies)
display(website_products)

## (2) Now loop through the first CSV and match to second CSV
The match, similar to a VLOOKUP, happens through a specified matching columns. In this case it is "Company" and "Website."

In [ ]:
# Loop through each row in the first CSV and insert the values of the rows for columns x, y, and z into the prompt
for index, row in companies.iterrows():
    company_in_row = row['Company']
    website_products_values = website_products['Website'].values.tolist()
    prompt = f"Using the best of your knowledge and precise data analysis skills, for the value {company_in_row}, please choose the most appropriate value from the list below. Return only the exact value as given with no additional text. Important, if you do not see a fitting result, return N/A. Values are: {website_products_values}"
    
    # Call the OpenAI API to generate a response
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=.1
    )

    # Extract the generated response from the API response
    generated_text = response.choices[0].message.content

    companies.loc[index, 'Website'] = generated_text
display(companies)


## (3) Now just join the datasets and return table

In [ ]:
companies_joined_with_website_products = pd.merge(companies, website_products, on='Website', how='left')
display(companies_joined_with_website_products)

## (4) Save the datasets as a file


In [ ]:
from save_file import save_new_csv
save_new_csv(file_path, companies_joined_with_website_products)

## (5) Fully built as a callable function
Call the function as follows:

```python
from llm_lookup import lookup

result = lookup(
    table1, # Pandas dataframe, required
    table2, # Pandas dataframe, required
    column_in_table1, # string, required
    column_in_table2, # string, required
    matching_context='', # string, optional
    openai_model='gpt-3.5-turbo', # string, optional
    temperature=0.1, # float, optional
    chunk_size=50, # int, optional
    return_stats=False # bool, optional
)
```

The optional parameters allow you to adjust the lookup for more tailored needs. You can choose the additional context, model, temperature, chunk size and whether or not to return stats on the lookup calls. The model, temperature and chunk size are an interplay that strongly depend on one another and the dataset given. Worth testing on a smaller data set and finetuning to see what works best for needs.

### (5.1) Example with just required variables

In [ ]:
from llm_lookup import lookup
merged_df = lookup(companies,website_products,'Company','Website')

Now show me the results:

In [ ]:
display(merged_df)

### (5.2) Example with optional variables
If you want to return stats, you also need to store them in a variable. 

In [ ]:
from llm_lookup import lookup
merged_df, stats = lookup(companies,website_products,'Company','Website',
                          matching_context='Given a company name, you are to match the correct URL website as provided',
                          openai_model='gpt-4o',
                          temperature=0.2,
                          chunk_size=3,
                          return_stats=True)

Show me the results as well as stats:

In [ ]:
display(merged_df)
display(stats)

Save the file if you want:

In [ ]:
from save_file import save_new_csv
save_new_csv(file_path, merged_df)